# LastFM Recommender Genre Topic Modeling: pyLDAvis
July [X], 2018

In [1]:
import sys
sys.executable

'/Users/brianmcmahon/anaconda3/envs/recommender/bin/python'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from os import path, getcwd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE

from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS

from PIL import Image

import gensim
from gensim import matutils, models

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

CWD = '/Volumes/BC_Clutch/Dropbox/recommender_system/'

In [3]:
# sets output to three decimals
pd.set_option('display.float_format',lambda x: '%.3f' %x)

In [4]:
tag_data = pd.read_csv('../data/recsys_tags2.csv')
tag_data = tag_data.set_index(['idx'])
tag_data['tags'] = tag_data['tags'].apply(lambda x: ', '.join(x.split('|')))
tag_data.head()

,name,tags
idx,,
0,3 Doors Down,"00s, 2008, 90s, acoustic, acoustic rock, alter..."
1,30 Seconds to Mars,"00s, <3, acoustic, addictive, alternative, alt..."
2,3OH!3,"00s, 10s, 2007, 2008, 2009, 2010, <3, addictiv..."
3,50 Cent,"00s, american, awesome, catchy, club, dance, g..."
4,A Day to Remember,"<3, acoustic, alternative, american, christian..."


In [5]:
tag_data.tags[0]

'00s, 2008, 90s, acoustic, acoustic rock, alternative, alternative rock, american, awesome, cute, grunge, guilty pleasure, hard rock, love, male vocalists, melancholic, other great tracks, pop, pop rock, post-grunge, prejebeno, punk, rock, romantic, soft rock, soundtrack'

In [6]:
tag_data.shape

(505, 2)

### Assumptions

In [7]:
data = tag_data.tags
n_topics = 6
seed = 42
ngram_range = (1,3)
ncols = 2

sw = set(STOPWORDS)

adds = ['rock','pop','hardcore','vocalist','male','female']
for word in adds:
    sw.add(word)


### pyLDAvis

In [66]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(data)
print(dtm_tf.shape)

(505, 264)


In [67]:
lda_tf = LatentDirichletAllocation(n_topics=n_topics, random_state=seed)
lda_tf.fit(dtm_tf)

/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=6,
             perp_tol=0.1, random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [68]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=           x      y  topics  cluster   Freq
topic                                      
4     -0.086  0.109       1        1 28.551
3      0.074  0.059       2        1 22.749
2     -0.184  0.094       3        1 14.102
1     -0.029 -0.094       4        1 12.988
0      0.264  0.044       5        1 12.928
5     -0.038 -0.212       6        1  8.683, topic_info=     Category    Freq          Term   Total  loglift  logprob
term                                                         
160   Default 391.000         metal 391.000   30.000   30.000
85    Default 415.000        female 415.000   29.000   29.000
128   Default 462.000         indie 462.000   28.000   28.000
168   Default 157.000           new 157.000   27.000   27.000
120   Default 140.000           hop 140.000   26.000   26.000
192   Default 247.000          punk 247.000   25.000   25.000
260   Default 117.000          wave 117.000   24.000   24.000
118   Default 106.000           hip 106.000   23.000   23.000
114   Default 122.000          hard 122.000   22.000   22.000
46    Default 206.000         dance 206.000   21.000   21.000
117   Default  59.000         heavy  59.000   20.000   20.000
64    Default 188.000    electronic 188.000   19.000   19.000
189   Default  98.000   progressive  98.000   18.000   18.000
6     Default  72.000       ambient  72.000   17.000   17.000
115   Default  99.000      hardcore  99.000   16.000   16.000
75    Default 105.000  experimental 105.000   15.000   15.000
239   Default  61.000      synthpop  61.000   14.000   14.000
184   Default 144.000          post 144.000   13.000   13.000
30    Default 122.000        catchy 122.000   12.000   12.000
26    Default 134.000       british 134.000   11.000   11.000
62    Default  99.000       electro  99.000   10.000   10.000
238   Default  38.000         synth  38.000    9.000    9.000
65    Default  89.000   electronica  89.000    8.000    8.000
199   Default  61.000         remix  61.000    7.000    7.000
132   Default  95.000  instrumental  95.000    6.000    6.000
48    Default  40.000         death  40.000    5.000    5.000
200   Default  77.000           rnb  77.000    4.000    4.000
201   Default  67.000          roll  67.000    3.000    3.000
67    Default  90.000           emo  90.000    2.000    2.000
191   Default  72.000   psychedelic  72.000    1.000    1.000
...       ...     ...           ...     ...      ...      ...
168    Topic6  93.322           new 157.166    1.923   -2.615
248    Topic6   9.892        trance  17.012    1.902   -4.859
241    Topic6  14.189        techno  24.452    1.899   -4.498
39     Topic6   7.957       college  14.119    1.870   -5.077
79     Topic6  11.755         faves  21.617    1.835   -4.687
52     Topic6  15.825         disco  31.050    1.770   -4.389
16     Topic6  12.889         bands  25.513    1.761   -4.595
71     Topic6   6.276       england  12.442    1.759   -5.314
199    Topic6  26.749         remix  61.809    1.606   -3.865
99     Topic6   4.781        german  11.667    1.552   -5.586
130    Topic6  12.752    industrial  31.659    1.534   -4.605
105    Topic6  15.601        gothic  42.296    1.446   -4.404
26     Topic6  44.897       british 134.958    1.343   -3.347
194    Topic6   6.428          rain  19.685    1.325   -5.290
228    Topic6   6.068         space  21.275    1.189   -5.348
65     Topic6  24.986   electronica  89.769    1.165   -3.933
62     Topic6  25.954       electro  99.727    1.098   -3.895
122    Topic6   9.991         house  39.559    1.068   -4.849
132    Topic6  23.703  instrumental  95.755    1.048   -3.985
64     Topic6  38.976    electronic 188.732    0.866   -3.488
46     Topic6  32.270         dance 206.757    0.586   -3.677
81     Topic6  25.226     favorites 173.809    0.514   -3.923
202    Topic6  16.017      romantic  86.126    0.762   -4.377
80     Topic6  21.313      favorite 210.754    0.152   -4.092
75     Topic6  14.669  experimental 105.547    0.470   -4.465
151    To

In [69]:
# 1. teen pop
# 2. punk
# 3. hip hop
# 4. electronic
# 5. heavy metal
# 6. electronic

In [70]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=           x      y  topics  cluster   Freq
topic                                      
4     -0.101  0.085       1        1 28.551
3      0.177  0.009       2        1 22.749
2     -0.203  0.041       3        1 14.102
1      0.089 -0.161       4        1 12.988
0      0.150  0.249       5        1 12.928
5     -0.111 -0.223       6        1  8.683, topic_info=     Category    Freq          Term   Total  loglift  logprob
term                                                         
160   Default 391.000         metal 391.000   30.000   30.000
85    Default 415.000        female 415.000   29.000   29.000
128   Default 462.000         indie 462.000   28.000   28.000
168   Default 157.000           new 157.000   27.000   27.000
120   Default 140.000           hop 140.000   26.000   26.000
192   Default 247.000          punk 247.000   25.000   25.000
260   Default 117.000          wave 117.000   24.000   24.000
118   Default 106.000           hip 106.000   23.000   23.000
114   Default 122.000          hard 122.000   22.000   22.000
46    Default 206.000         dance 206.000   21.000   21.000
117   Default  59.000         heavy  59.000   20.000   20.000
64    Default 188.000    electronic 188.000   19.000   19.000
189   Default  98.000   progressive  98.000   18.000   18.000
6     Default  72.000       ambient  72.000   17.000   17.000
115   Default  99.000      hardcore  99.000   16.000   16.000
75    Default 105.000  experimental 105.000   15.000   15.000
239   Default  61.000      synthpop  61.000   14.000   14.000
184   Default 144.000          post 144.000   13.000   13.000
30    Default 122.000        catchy 122.000   12.000   12.000
26    Default 134.000       british 134.000   11.000   11.000
62    Default  99.000       electro  99.000   10.000   10.000
238   Default  38.000         synth  38.000    9.000    9.000
65    Default  89.000   electronica  89.000    8.000    8.000
199   Default  61.000         remix  61.000    7.000    7.000
132   Default  95.000  instrumental  95.000    6.000    6.000
48    Default  40.000         death  40.000    5.000    5.000
200   Default  77.000           rnb  77.000    4.000    4.000
201   Default  67.000          roll  67.000    3.000    3.000
67    Default  90.000           emo  90.000    2.000    2.000
191   Default  72.000   psychedelic  72.000    1.000    1.000
...       ...     ...           ...     ...      ...      ...
168    Topic6  93.322           new 157.166    1.923   -2.615
248    Topic6   9.892        trance  17.012    1.902   -4.859
241    Topic6  14.189        techno  24.452    1.899   -4.498
39     Topic6   7.957       college  14.119    1.870   -5.077
79     Topic6  11.755         faves  21.617    1.835   -4.687
52     Topic6  15.825         disco  31.050    1.770   -4.389
16     Topic6  12.889         bands  25.513    1.761   -4.595
71     Topic6   6.276       england  12.442    1.759   -5.314
199    Topic6  26.749         remix  61.809    1.606   -3.865
99     Topic6   4.781        german  11.667    1.552   -5.586
130    Topic6  12.752    industrial  31.659    1.534   -4.605
105    Topic6  15.601        gothic  42.296    1.446   -4.404
26     Topic6  44.897       british 134.958    1.343   -3.347
194    Topic6   6.428          rain  19.685    1.325   -5.290
228    Topic6   6.068         space  21.275    1.189   -5.348
65     Topic6  24.986   electronica  89.769    1.165   -3.933
62     Topic6  25.954       electro  99.727    1.098   -3.895
122    Topic6   9.991         house  39.559    1.068   -4.849
132    Topic6  23.703  instrumental  95.755    1.048   -3.985
64     Topic6  38.976    electronic 188.732    0.866   -3.488
46     Topic6  32.270         dance 206.757    0.586   -3.677
81     Topic6  25.226     favorites 173.809    0.514   -3.923
202    Topic6  16.017      romantic  86.126    0.762   -4.377
80     Topic6  21.313      favorite 210.754    0.152   -4.092
75     Topic6  14.669  experimental 105.547    0.470   -4.465
151    To

In [71]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

/Users/brianmcmahon/anaconda3/envs/recommender/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            x       y  topics  cluster   Freq
topic                                        
4      93.013  41.924       1        1 28.551
3      -7.257 -72.205       2        1 22.749
2       6.779  79.065       3        1 14.102
1     -55.191   8.529       4        1 12.988
0      84.338 -51.566       5        1 12.928
5      24.337   1.149       6        1  8.683, topic_info=     Category    Freq          Term   Total  loglift  logprob
term                                                         
160   Default 391.000         metal 391.000   30.000   30.000
85    Default 415.000        female 415.000   29.000   29.000
128   Default 462.000         indie 462.000   28.000   28.000
168   Default 157.000           new 157.000   27.000   27.000
120   Default 140.000           hop 140.000   26.000   26.000
192   Default 247.000          punk 247.000   25.000   25.000
260   Default 117.000          wave 117.000   24.000   24.000
118   Default 106.000           hip 106.000   23.000   23.000
114   Default 122.000          hard 122.000   22.000   22.000
46    Default 206.000         dance 206.000   21.000   21.000
117   Default  59.000         heavy  59.000   20.000   20.000
64    Default 188.000    electronic 188.000   19.000   19.000
189   Default  98.000   progressive  98.000   18.000   18.000
6     Default  72.000       ambient  72.000   17.000   17.000
115   Default  99.000      hardcore  99.000   16.000   16.000
75    Default 105.000  experimental 105.000   15.000   15.000
239   Default  61.000      synthpop  61.000   14.000   14.000
184   Default 144.000          post 144.000   13.000   13.000
30    Default 122.000        catchy 122.000   12.000   12.000
26    Default 134.000       british 134.000   11.000   11.000
62    Default  99.000       electro  99.000   10.000   10.000
238   Default  38.000         synth  38.000    9.000    9.000
65    Default  89.000   electronica  89.000    8.000    8.000
199   Default  61.000         remix  61.000    7.000    7.000
132   Default  95.000  instrumental  95.000    6.000    6.000
48    Default  40.000         death  40.000    5.000    5.000
200   Default  77.000           rnb  77.000    4.000    4.000
201   Default  67.000          roll  67.000    3.000    3.000
67    Default  90.000           emo  90.000    2.000    2.000
191   Default  72.000   psychedelic  72.000    1.000    1.000
...       ...     ...           ...     ...      ...      ...
168    Topic6  93.322           new 157.166    1.923   -2.615
248    Topic6   9.892        trance  17.012    1.902   -4.859
241    Topic6  14.189        techno  24.452    1.899   -4.498
39     Topic6   7.957       college  14.119    1.870   -5.077
79     Topic6  11.755         faves  21.617    1.835   -4.687
52     Topic6  15.825         disco  31.050    1.770   -4.389
16     Topic6  12.889         bands  25.513    1.761   -4.595
71     Topic6   6.276       england  12.442    1.759   -5.314
199    Topic6  26.749         remix  61.809    1.606   -3.865
99     Topic6   4.781        german  11.667    1.552   -5.586
130    Topic6  12.752    industrial  31.659    1.534   -4.605
105    Topic6  15.601        gothic  42.296    1.446   -4.404
26     Topic6  44.897       british 134.958    1.343   -3.347
194    Topic6   6.428          rain  19.685    1.325   -5.290
228    Topic6   6.068         space  21.275    1.189   -5.348
65     Topic6  24.986   electronica  89.769    1.165   -3.933
62     Topic6  25.954       electro  99.727    1.098   -3.895
122    Topic6   9.991         house  39.559    1.068   -4.849
132    Topic6  23.703  instrumental  95.755    1.048   -3.985
64     Topic6  38.976    electronic 188.732    0.866   -3.488
46     Topic6  32.270         dance 206.757    0.586   -3.677
81     Topic6  25.226     favorites 173.809    0.514   -3.923
202    Topic6  16.017      romantic  86.126    0.762   -4.377
80     Topic6  21.313      favorite 210.754    0.152   -4.092
75     Topic6  14.669  experimental 105.547    0.470   

In [72]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [73]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

data_vectorized = vectorizer.fit_transform(tag_data.tags)

In [74]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(data))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [75]:
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)